# Lab 4 – Predicting a Continuous Target with Regression (Titanic)
#### Nick Elias
#### Date: 04/01/2025

### Introduction:

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Section 1. Import and Inspect the Data
Load the Titanic dataset and confirm it’s structured correctly.

In [2]:
# Load Titanic dataset from seaborn and verify
titanic = sns.load_dataset("titanic")
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Section 2. Data Exploration and Preparation
Prepare the Titanic data for regression modeling. See the previous work.

- Impute missing values for age using median
- Drop rows with missing fare (or impute if preferred)
- Create numeric variables (e.g., family_size from sibsp + parch + 1)
    - Optional - convert categorical features (e.g. sex, embarked) if you think they might help your prediction model. (We do not know relationships until we evaluate things.)

In [3]:
titanic['age'].fillna(titanic['age'].median(), inplace=True)

titanic = titanic.dropna(subset=['fare'])

titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1

C:\Users\drumm\AppData\Local\Temp\ipykernel_7068\1712063472.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic['age'].fillna(titanic['age'].median(), inplace=True)


## Section 3. Feature Selection and Justification
Define multiple combinations of features to use as inputs to predict fare.

Use unique names (X1, y1, X2, y2, etc.) so results are visible and can be compared at the same time. 

Remember the inputs, usually X, are a 2D array. The target is a 1D array. 

In [4]:
# Case 1. age
X1 = titanic[['age']]
y1 = titanic['fare']

In [5]:
# Case 2. family_size
X2 = titanic[['family_size']]
y2 = titanic['fare']

In [6]:
# Case 3. age, family_size
X3 = titanic[['age', 'family_size']]
y3 = titanic['fare']

In [7]:
# Case 4. ???
#X4 = titanic[[???]]
#y4 = titanic['fare']

## Reflection Questions - answer these in your notebook (in a Markdown cell):

- Why might these features affect a passenger’s fare:
- List all available features:
- Which other features could improve predictions and why:
- How many variables are in your Case 4:
- Which variable(s) did you choose for Case 4 and why do you feel those could make good inputs: 

## Section 4. Train a Regression Model (Linear Regression)

#### 4.1 Split the Data

In [8]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=123)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=123)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=123)

X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.2, random_state=123)

NameError: name 'X4' is not defined

#### 4.2 Train and Evaluate Linear Regression Models (all 4 cases)

In [ ]:
lr_model1 = LinearRegression().fit(X1_train, y1_train)
lr_model2 = LinearRegression().fit(X2_train, y2_train)
lr_model3 = LinearRegression().fit(X3_train, y3_train)
lr_model4 = LinearRegression().fit(X4_train, y4_train)

# Predictions

y_pred_train1 = lr_model1.predict(X1_train)
y_pred_test1 = lr_model1.predict(X1_test)

y_pred_train2 = lr_model2.predict(X2_train)
y_pred_test2 = lr_model2.predict(X2_test)

# TODO: repeat for case 3 and 4 .... 

#### 4.3 Report Performance

In [ ]:
print("Case 1: Training R²:", r2_score(y1_train, y1_pred_train))
print("Case 1: Test R²:", r2_score(y1_test, y1_pred_test))
print("Case 1: Test RMSE:", mean_squared_error(y1_test, y1_pred_test, squared=False))
print("Case 1: Test MAE:", mean_absolute_error(y1_test, y1_pred_test))

# TODO: Repeat for Cases 2-4....

## Section 4 Reflection Questions - answer these in your notebook (in a Markdown cell):

#### Compare the train vs test results for each.

1. Did Case 1 overfit or underfit? Explain:
2. Did Case 2 overfit or underfit? Explain:
3. Did Case 3 overfit or underfit? Explain:
4. Did Case 4 overfit or underfit? Explain:

* Adding Age

1. Did adding age improve the model:
2. Propose a possible explanation (consider how age might affect ticket price, and whether the data supports that): 

* Worst

1. Which case performed the worst:
2. How do you know: 
3. Do you think adding more training data would improve it (and why/why not): 

* Best

1. Which case performed the best:
2. How do you know: 
3. Do you think adding more training data would improve it (and why/why not): 

## Section 5. Compare Alternative Models

In this section, we will take the best-performing case and explore other regression models.

#### Choose Best Case to Continue
Choose the best case model from the four cases. Use that model to continue to explore additional continuous prediction models. The following assumes that Case 1 was the best predictor  - this may not be the case. Adjust the code to use your best case model instead. 

#### Choosing Options
When working with regression models, especially those with multiple input features, we may run into overfitting — where a model fits the training data too closely and performs poorly on new data. To prevent this, we can apply regularization.

Regularization adds a penalty to the model’s loss function, discouraging it from using very large weights (coefficients). This makes the model simpler and more likely to generalize well to new data.

In general: 
* If the basic linear regression is overfitting, try Ridge.

* If you want the model to automatically select the most important features, try Lasso.

* If you want a balanced approach, try Elastic Net.

### 5.1 Ridge Regression (L2 penalty)

Ridge Regression is a regularized version of linear regression that adds a penalty to large coefficient values. It uses the L2 penalty, which adds the sum of squared coefficients to the loss function.

This "shrinks" the coefficients, reducing the model’s sensitivity to any one feature while still keeping all features in the model.

* Penalty term: L2 = sum of squared weights
* Effect: Shrinks weights, helps reduce overfitting, keeps all features

In [ ]:
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X1_train, y1_train)
y_pred_ridge = ridge_model.predict(X1_test)

### 5.2 Elastic Net (L1 + L2 combined)

Lasso Regression uses the L1 penalty, which adds the sum of absolute values of the coefficients to the loss function. Lasso can shrink some coefficients all the way to zero, effectively removing less important features. This makes it useful for feature selection.

* Penalty term: L1 = sum of absolute values of weights
* Effect: Can shrink some weights to zero (drops features), simplifies the model

Elastic Net combines both L1 (Lasso) and L2 (Ridge) penalties. It balances the feature selection ability of Lasso with the stability of Ridge.

We control the balance with a parameter called l1_ratio:
* If l1_ratio = 0, it behaves like Ridge
* If l1_ratio = 1, it behaves like Lasso
* Values in between mix both types
* Penalty term: α × (L1 + L2)
* Effect: Shrinks weights and can drop some features — flexible and powerful

In [ ]:
elastic_model = ElasticNet(alpha=0.3, l1_ratio=0.5)
elastic_model.fit(X1_train, y1_train)
y_pred_elastic = elastic_model.predict(X1_test)

### 5.3 Polynomial Regression

Linear regression is a simple two dimensional relationship - a simple straight line. But we can test more complex relationships. Polynomial regression adds interaction and nonlinear terms to the model. Be careful here - higher-degree polynomials can easily overfit.

In [ ]:
# Set up the poly inputs
poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X1_train)
X_test_poly = poly.transform(X1_test)

In [ ]:
# Use the poly inputs in the LR model
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y1_train)
y_pred_poly = poly_model.predict(X1_test_poly)

### 5.4 Visualize Polynomial Cubic Fit (for 1 input feature)

Choose a case with just one input feature and plot it. For example:

In [ ]:
plt.scatter(X1_test, y1_test, color='blue', label='Actual')
plt.scatter(X1_test, y_pred_poly, color='red', label='Predicted (Poly)')
plt.legend()
plt.title("Polynomial Regression: Age vs Fare")
plt.show()

### 5.4 Reflections (in a Markdown cell):

1. What patterns does the cubic model seem to capture:
2. Where does it perform well or poorly:
3. Did the polynomial fit outperform linear regression:
4. Where (on the graph or among which kinds of data points) does it fit best:

### 5.5 Compare All Models

Create a summary table or printout comparing all models:

In [ ]:
def report(name, y_true, y_pred):
    print(f"{name} R²: {r2_score(y_true, y_pred):.3f}")
    print(f"{name} RMSE: {mean_squared_error(y_true, y_pred, squared=False):.2f}")
    print(f"{name} MAE: {mean_absolute_error(y_true, y_pred):.2f}\n")

report("Linear", y1_test, y1_pred_test)
report("Ridge", y1_test, y_pred_ridge)
report("ElasticNet", y1_test, y_pred_elastic)
report("Polynomial", y1_test, y_pred_poly)

### 5.5 Visualize Higher Order Polynomial (for the same 1 input case)

Use the same single input case as you visualized above, but use a higher degree polynomial (e.g. 4, 5, 6, 7, or 8). Plot the result. 

In a Markdown cell, tell us which option seems to work better - your initial cubic (3) or your higher order and why. 

## Section 6. Final Thoughts & Insights

Your notebook should tell a data story. Use this section to demonstrate your thinking and value as an analyst.

### 6.1 Summarize Findings
* What features were most useful?

* What regression model performed best?

* How did model complexity or regularization affect results?

### 6.2 Discuss Challenges
* Was fare hard to predict? Why?

* Did skew or outliers impact the models?

### 6.3 Optional Next Steps
Try different features besides the ones used (e.g., pclass, sex if you didn't use them this time)

Try predicting age instead of fare

Explore log transformation of fare to reduce skew